In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

data_path = "/content/drive/MyDrive/NLP Project/quoref-train-dev-v0.1.zip"
zip_ref = zipfile.ZipFile(data_path, 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.3MB 6.8MB/s 
     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 901kB 18.7MB/s 


In [ ]:
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 512
configuration = BertConfig()  # default parameters and configuration for BERT

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

     |████████████████████████████████| 706kB 6.2MB/s 


In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # using the pretrained tokenizer
save_path = "/content/bert_base_uncased/"
#if not os.path.exists(save_path):
    #os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("/content/bert_base_uncased/vocab.txt", lowercase=True)


# loading the data

train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path = keras.utils.get_file("eval.json", eval_data_url)

# train_path = '/content/quoref-train-dev-v0.1/quoref-train-v0.1.json'
# eval_path = '/content/quoref-train-dev-v0.1/quoref-dev-v0.1.json'

with open(train_path) as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)


4857856/4854279 [==============================] - 0s 0us/step


In [ ]:
class DataInstance:

  def __init__(self, question, context, start_word_idx, answer_text, all_answers):
    self.question = question
    self.context = context
    self.start_word_idx = start_word_idx
    self.answer_text = answer_text
    self.all_answers = all_answers
    self.skip = False


  def preprocess(self):
    context = self.context
    question = self.question
    answer_text = self.answer_text
    all_answers = self.all_answers
    start_word_idx = self.start_word_idx
    end_word_idx = []

    # clean context, answer and question
    context = " ".join(str(context).split())
    question = " ".join(str(question).split())
    answer = " ".join(str(answer_text).split())

    # Find end character index of answer in context
    for i in range(len(all_answers)):
        e = start_word_idx[i] + len(all_answers[i])
        if (e >= len(context)):
            self.skip = True
            return
        
        end_word_idx.append(e)    


    # Mark the word indices in context that are in answer
    is_word_in_ans = [0 for _ in range(len(context))]
    
    for i in range(len(start_word_idx)):
        for idx in range(start_word_idx[i], end_word_idx[i]):
            is_word_in_ans[idx] = 1

    # Tokenize context
    tokenized_context = tokenizer.encode(context)


    # Find the words that were created from answer characters
    # The offsets associated to each token. 
    # These offsets let’s you slice the input string, and thus 
    # retrieve the original part that led to producing the corresponding token.
    ans_token_idx = []
    for idx, (start, end) in enumerate(tokenized_context.offsets):
      if (sum(is_word_in_ans[start:end]) > 0):
        ans_token_idx.append(idx)


    if len(ans_token_idx)==0:
      self.skip = True
      return

    label_i = [0] * 512
    label_o = [1] * 512

    for i in ans_token_idx:
        if(i >= 512):
            self.skip = True
            return
        label_i[i] = 1
        label_o[i] = 0

    # tokenize question 
    tokenized_question = tokenizer.encode(question)

    # Create inputs
    #  The generated ID 
    #The IDs are the main input to a Language Model. 
    #They are the token indices, the numerical representations that a LM understands.
    input_ids = tokenized_context.ids + tokenized_question.ids[1:]
    token_type_ids = [0 for _ in  range(len(tokenized_context.ids))] + [1 for _ in range(len(tokenized_question.ids[1:]))]
    attention_mask = [1 for _ in range(len(input_ids))]


    # Pad and create attention masks.
    # Skip if truncation is needed
    padding_length = max_len - len(input_ids)
    if padding_length > 0:  # pad
        input_ids = input_ids + ([0 for _ in range(padding_length)])
        attention_mask = attention_mask + ([0 for _ in range(padding_length)])
        token_type_ids = token_type_ids + ([0 for _ in range(padding_length)])
    elif padding_length < 0:  # skip
        self.skip = True
        return

    self.input_ids = input_ids
    self.token_type_ids = token_type_ids
    self.attention_mask = attention_mask
    self.label_i = label_i
    self.label_o = label_o
    self.context_token_to_word = tokenized_context.offsets

In [ ]:
raw_eval_data["data"][0]["paragraphs"][0]["context"][208]

'i'

In [ ]:
def create_instance(raw_data):
  data_instances = []
  for item in raw_data["data"]:
    for para in item["paragraphs"]:
      context = para["context"]
      for qa in para["qas"]:
        question = qa["question"]
        answer_text = qa["answers"][0]["text"] # considering only first answer
        all_answers = [ans["text"] for ans in qa["answers"]]
        start_char_idx = qa["answers"][0]["answer_start"]
        all_start_idx = [ans["answer_start"] for ans in qa["answers"]]
        quoref_instance = DataInstance(question, context, all_start_idx, answer_text, all_answers)
        quoref_instance.preprocess()
        data_instances.append(quoref_instance)
  
  return data_instances


def create_input_targets(data_instances):
  dataset_dict = {
      "input_ids" : [],
      "token_type_ids" : [],
      "attention_mask" : [],
      "label_i" : [],
      "label_o" : []
  }

  for item in data_instances:
    if item.skip == False:
      for key in dataset_dict:
        dataset_dict[key].append(getattr(item, key))
  for key in dataset_dict:
    dataset_dict[key] = np.array(dataset_dict[key])

  x = [
       dataset_dict["input_ids"],
       dataset_dict["token_type_ids"],
       dataset_dict["attention_mask"],
  ]
  y = [
       dataset_dict["label_i"],
       dataset_dict["label_o"]
  ]

  return x, y

In [ ]:
train_instances = create_instance(raw_train_data)
x_train, y_train = create_input_targets(train_instances)
print("trainset size: {}".format(len(train_instances)))

eval_instances = create_instance(raw_eval_data)
x_eval, y_eval = create_input_targets(eval_instances)
print("evalset size: {}".format(len(eval_instances)))

trainset size: 87599
evalset size: 10570


In [ ]:
len(y_train[0][0])

512

In [ ]:
def create_model():
  
  encoder = TFBertModel.from_pretrained("bert-base-uncased")

  # single span QA model
  input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
  token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
  attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
  embedding = encoder(
                input_ids = input_ids,
                token_type_ids = token_type_ids,
                attention_mask = attention_mask,
              )[0] # outputs last hidden state
  
#   layer1 = layers.Dense(2, name="layer1")(embedding)
#   layer1 = layers.Activation(keras.activations.relu)(layer1)
#   layer1 = layers.Flatten()(layer1)
# #   layer1 = layers.Dropout(0.2)(layer1)
#   last_layer = layers.Dense(1024, name="last_layer" )(layer1)
  last_layer = layers.Dense(2, name="last_layer")(embedding)
  last_layer = layers.Activation(keras.activations.softmax)(last_layer) 


  model = keras.Model(
      inputs = [input_ids, token_type_ids, attention_mask],
      outputs = [last_layer]
  )

  l = keras.losses.SparseCategoricalCrossentropy(from_logits=False) # cross entropy on probs of last layer
#   loss = keras.losses.binary_crossentropy()
  #optimizer = keras.optimizers.Adam(lr=5e-5)
#   optimizer = tfa.optimizers.SGDW(weight_decay=0.01,learning_rate = 5e-5 )
  optimizer = tfa.optimizers.AdamW(weight_decay=0.01, learning_rate = 5e-5)
  model.compile(optimizer=optimizer, loss=l)

  return model

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.19.39.2:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.19.39.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 109482240   input_4[0][0]                    
                                                                 input_6[0][0]              

In [ ]:
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text

class ExactMatch(keras.callbacks.Callback):
  def __init__(self, x_eval, y_eval):
    self.x_eval = x_eval
    self.y_eval = y_eval

  def on_epoch_end(self, epoch, logs=None):
    pred_start, pred_end = self.model.predict(self.x_eval)
    count = 0
    eval_instances_no_skip = [ele for ele in eval_instances if ele.skip==False]
    
    for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
      quoref_eg = eval_instances_no_skip[idx]
      offsets = quoref_eg.context_token_to_word
      mat = np.outer(start, end)
      pos = np.argmax(mat)
      start = pos//mat.shape[1]
      end = pos%mat.shape[1]
      if (start >= len(offsets)):
        continue
      pred_word_start = offsets[start][0]
      if (end < len(offsets)):
        pred_word_end = offsets[end][1]
        pred_ans = quoref_eg.context[pred_word_start:pred_word_end] # span of predicted ans with their indices # pred_word_start is the starting index, pred_word_end is the ending index in context
      else:
        pred_ans = quoref_eg.context[pred_word_start:]

      normalized_pred_ans = normalize_text(pred_ans)
      normalized_true_ans = [normalize_text(_) for _ in quoref_eg.all_answers]
      if (normalized_pred_ans in normalized_true_ans):
        count += 1
    acc_whole = count/len(self.y_eval[0])
    acc  = count/len(eval_instances_no_skip)
    print(f"epoch={epoch+1}, EM_on_whole = {acc_whole:.2f}, exact match score={acc:.2f}\n")


In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
history = model.fit(
    x_train,
    y_train,
    epochs=1,
    verbose=2,
    batch_size=64,
    # callbacks=[exact_match_callback],
)

1358/1358 - 361s - loss: 0.5416


In [ ]:
history.history["loss"]

[0.019134169444441795]

In [ ]:
x_t = []
x_t.append(x_train[0][0:1000])
x_t.append(x_train[1][0:1000])
x_t.append(x_train[2][0:1000])

y_t = []
y_t.append(y_train[0][0:1000])
y_t.append(y_train[1][0:1000])



In [ ]:
x_test = []
x_test.append(x_eval[0][2:3])
x_test.append(x_eval[1][2:3])
x_test.append(x_eval[2][2:3])

y_test = []
y_test.append(y_eval[0][2:3])
y_test.append(y_eval[1][2:3])


In [ ]:
temp = model.predict(x_t)

In [ ]:
temp[0][43:46]

array([[9.9995267e-01, 4.7293335e-05],
       [9.9990737e-01, 9.2667622e-05],
       [9.9996400e-01, 3.5967820e-05]], dtype=float32)

In [ ]:
p.shape

NameError: ignored

In [ ]:

temp = model.predict(x_t)

preds = []
for i in range(temp.shape[0]):
    t = []
    for j in range(temp.shape[1]):
        if(temp[i][j][0] > temp[i][j][1]):
            t.append(0)
        else:
            t.append(1) 
    preds.append(t)
p = np.array(preds)

correct = 0
wrong = 0
for i in range(1000):

    if(np.array_equal(p[i],y_t[0][i])):
        correct += 1
    else:
        wrong += 1

print(correct, wrong, correct/(correct + wrong))

0 1000 0.0


In [ ]:
y_eval[0][0]

In [ ]:
p[0]

In [ ]:
x_eval[0].shape

In [ ]:
preds = []
for i in range(temp.shape[0]):
    t = []
    for j in range(temp.shape[1]):
        if(temp[i][j][0] > temp[i][j][1]):
            t.append(0)
        else:
            t.append(1) 
    preds.append(t)
p = np.array(preds)

np.array_equal(p[0],y_test[0])

In [ ]:
p[0]

In [ ]:
correct = 0
wrong = 0
for i in range(1662):

    if(np.array_equal(p[i],y_eval[0][i])):
        correct += 1
    else:
        wrong += 1

print(correct, wrong, correct/(correct + wrong))

In [ ]:
temp[0][93:99]


In [ ]:
y_test[0][0][93:99]

In [ ]:
for i in range(len(y_test[0][0])):
    if y_test[0][0][i] == 1:
        print(i)

In [ ]:
y_test[0][0][45]

In [ ]:
# serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")


# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")

In [ ]:
# saving the model

# model.save('Single_span_BERT_model.h5')
# loaded_model = keras.models.load_model('/content/Single_span_BERT_model.h5')